In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import string

In [2]:
filename='alice.txt'
f=open(filename,'r',encoding='utf-8')
file=f.read()
file=file.lower()
all_characters=sorted(list(set(file)))
print(all_characters)
n_characters=len(all_characters)
print(n_characters)

['\n', ' ', '!', '(', ')', '*', ',', '-', '.', '0', '3', ':', ';', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '‘', '’', '“', '”', '\ufeff']
48


In [3]:
file_len=len(file)
print(file_len)

144410


In [4]:
seq_length=200
# chunk_len=100

# def random_chunk():
#     start=np.random.randint(0,file_len-chunk_len)
#     end=start+chunk_len+1
#     return file[start:end]

# print(random_chunk())

In [5]:
class RNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,n_layers=1):
        super(RNN,self).__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.output_size=output_size
        self.n_layers=n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(self,input,hidden):
        batch_size=input.size(0)
        input=self.encoder(input)
        output,hidden=self.gru(input.view(1,batch_size,-1),hidden)
        output = self.decoder(output.view(batch_size, -1))
        return output, hidden
    
    def init_hidden(self,batch_size):
        return torch.zeros(self.n_layers,batch_size,self.hidden_size)

In [6]:
def char_tensor(string):
    tensor=torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c]=all_characters.index(string[c])
    return tensor

In [7]:
def random_training_set(seq_length, batch_size):
    
    input = torch.LongTensor(batch_size, seq_length)
    target = torch.LongTensor(batch_size, seq_length)
    for i in range(batch_size):
        start_idx = np.random.randint(0, file_len - seq_length)
        end_idx = start_idx + seq_length + 1
        seq = file[start_idx:end_idx]
        input[i] = char_tensor(seq[:-1])
        target[i] = char_tensor(seq[1:])
    return input, target

In [8]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden=charnet.init_hidden(1)
    prime_input=char_tensor(prime_str).unsqueeze(0)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        output, hidden = charnet(prime_input[:,p], hidden)
    inp = prime_input[:,-1]
    
    for p in range(predict_len):
        output, hidden = charnet(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char).unsqueeze(0)

    return predicted

In [9]:
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [10]:
def train(inp, target):
    hidden = charnet.init_hidden(batch_size)
    charnet.zero_grad()
    loss=0
    for c in range(seq_length):
        output, hidden =charnet(inp[:,c], hidden)
        loss += criterion(output.view(batch_size,-1),target[:,c])
    loss.backward()
    charnet_optimizer.step()

    return loss.item()/seq_length

In [11]:
n_epochs = 3000
print_every = 100
plot_every = 10
hidden_size = 100
n_layers = 1
lr = 0.001
batch_size=3

charnet=RNN(n_characters, hidden_size, n_characters, n_layers)
charnet_optimizer = torch.optim.Adam(charnet.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    loss = train(*random_training_set(seq_length,batch_size))       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(evaluate('wh', 100), '\n')

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

 26s (100 3%) 2.3524]
wheiat hal.

and bely we thoun ipe pitho aseererd ho  andf too in thenle the orout but ther rangk it!  

 52s (200 6%) 2.1839]
whe tthed tede own caring frot rengroust as wises, hep and bedt the sin or the thates wast bel do or s 

 17s (300 10%) 1.9515]
what os eanted at mep

on, and in dea lit, anking
y haid ther, thes the curerp walll sure hers
‘ight e 

 43s (400 13%) 1.9660]
whe a mare the davoos with alice and oht astoourting pleassing ‘at out in you bitted loond a one, and  

 10s (500 16%) 1.7854]
whowa. ‘cainge, and that.’ and, shery litter into it,’ said ale she ware up a sid quie mared a lexdele 

 39s (600 20%) 1.7799]
whil of in, the mody to hat alice woutly the dood in’t one, and at the ‘it thee thouger.’

‘didner and 

 8s (700 23%) 1.6821]
whid that and aspight, and the might have tras all! and paning, bus underst the
the beathing supp said 

 38s (800 26%) 1.6685]
whe of somen’t alice are the prallok! the dead look evertrild of the don’t not this 